In [161]:
import pandas as pd
import numpy as np
import re # regex for string matching

# Important Notes (READ BEFORE RUNNING CODE):
# I flagged code areas that need user interaction with *UPDATE HERE* so ctrl-F *UPDATE HERE* whenever you do a new upload
# Ctrl-F *UPDATE GIFT TRACKING* after you run the code to find the data to update in the Gift Data Tracking workbook
# At any point if you want to check the data in Excel, ctrl-F "to_csv" to find the CSV export you need. Uncomment it out, update the path & run the code.

# Process gift data one year at a time (let's do 2019 first)

# *UPDATE HERE* - every CSV import below

# 1. T3010 Donees dataset - delete all unnecessary columns first
donees = pd.read_csv(r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\T3010 - Donees - 2019 - CW - prepped.csv',
                    encoding='ANSI')

# 2. Master Reference Table - Gift Data from Gift Data Tracking Sheet
reference = pd.read_csv(r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\Gift Data Tracking - Master Reference Table 2019.csv')

# 3. Masterlist report from Contenta containing all foundations we have in GC
masterlist = pd.read_csv(r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\Contenta MasterList.csv')

# 4. CRA Charities - ALl
charities = pd.read_csv(r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\Charities - All.csv',
                           encoding='ANSI')

print (donees)

       BN/Registration number Donee Business number  \
0             100048800RR0001       118812627RR0001   
1             100048800RR0001                   NaN   
2             100048800RR0001       119038628RR0001   
3             100071927RR0001       737775486RR0001   
4             100072586RR0001        857802409RR001   
...                       ...                   ...   
265926        899977177RR0001       119108421RR0001   
265927        899977177RR0001             110245611   
265928        899977177RR0001       118801448RR0001   
265929        899990766RR0001       119306918RR0001   
265930        899990766RR0001       108021106RR0001   

                                    Donee Name           City  \
0                                   B'NAI ZION        TORONTO   
1                               ORTHODOX UNION            USA   
2                                      MEOROTH        TORONTO   
3       FONDATION AFS INTERCULTURE CANADA INC.       MONTRÉAL   
4             

c:\users\catherine\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [162]:
# Step 4

# Use this commented-out code if unnecessary columns haven't been deleted yet
# donees = pd.DataFrame(donees, columns= ['BN/Registration number', 'Donee Business number', 'Donee Name', 
#                                        'City', 'Total amount gifts'])

donees.rename(columns = {"BN/Registration number": "BN", "Donee Business number": "DoneeBN",
                        "Donee Name": "DoneeName", "Total amount gifts": "ReportedAmt"}, inplace = True)
print(donees)

                     BN          DoneeBN  \
0       100048800RR0001  118812627RR0001   
1       100048800RR0001              NaN   
2       100048800RR0001  119038628RR0001   
3       100071927RR0001  737775486RR0001   
4       100072586RR0001   857802409RR001   
...                 ...              ...   
265926  899977177RR0001  119108421RR0001   
265927  899977177RR0001        110245611   
265928  899977177RR0001  118801448RR0001   
265929  899990766RR0001  119306918RR0001   
265930  899990766RR0001  108021106RR0001   

                                     DoneeName           City  ReportedAmt  
0                                   B'NAI ZION        TORONTO      $450.00  
1                               ORTHODOX UNION            USA    $1,000.00  
2                                      MEOROTH        TORONTO      $880.00  
3       FONDATION AFS INTERCULTURE CANADA INC.       MONTRÉAL  $200,000.00  
4                  GLOBAL CENTRE FOR PLURALISM         OTTAWA    $3,204.00  
...      

In [163]:
# Add columns
# *UPDATE HERE* - Update Year accordingly
donees['Year'] = 2019
donees['Purpose'] = ""
# donees['Foundation Activity'] = [] - might not need to add now, will add in step 7

print(donees)

                     BN          DoneeBN  \
0       100048800RR0001  118812627RR0001   
1       100048800RR0001              NaN   
2       100048800RR0001  119038628RR0001   
3       100071927RR0001  737775486RR0001   
4       100072586RR0001   857802409RR001   
...                 ...              ...   
265926  899977177RR0001  119108421RR0001   
265927  899977177RR0001        110245611   
265928  899977177RR0001  118801448RR0001   
265929  899990766RR0001  119306918RR0001   
265930  899990766RR0001  108021106RR0001   

                                     DoneeName           City  ReportedAmt  \
0                                   B'NAI ZION        TORONTO      $450.00   
1                               ORTHODOX UNION            USA    $1,000.00   
2                                      MEOROTH        TORONTO      $880.00   
3       FONDATION AFS INTERCULTURE CANADA INC.       MONTRÉAL  $200,000.00   
4                  GLOBAL CENTRE FOR PLURALISM         OTTAWA    $3,204.00   
...

In [164]:
# Step 5 - Identify and remove any funder not in GC

# Rename masterlist Funder BN to match Donees BN column so we can join the two datasets
masterlist.rename(columns = {"BusinessNumber": "BN"}, inplace = True)
keep_bn = pd.DataFrame(masterlist, columns= ['BN'])


step_five = pd.merge(donees, keep_bn, on ='BN', how ='inner')
print(step_five)

                     BN          DoneeBN  \
0       100504679RR0001  100504687RR0001   
1       101015469RR0001  893081075RR0001   
2       101469609RR0001  118963081RR0001   
3       101469609RR0001  118846179RR0001   
4       101569697RR0001  119301034RR0001   
...                 ...              ...   
162401  899977177RR0001  870077757RR0001   
162402  899977177RR0001  107461485RR0001   
162403  899977177RR0001  119108421RR0001   
162404  899977177RR0001        110245611   
162405  899977177RR0001  118801448RR0001   

                                                DoneeName           City  \
0                                   IMPERIAL THEATRE INC.     SAINT JOHN   
1         KINCARDINE AND COMMUNITY HEALTH CARE FOUNDATION     KINCARDINE   
2         INSTITUT UNIVERSITAIRE EN SANTÉ MENTALE DOUGLAS       MONTRÉAL   
3                CENTRE DE RECHERCHE DE L'HÔPITAL DOUGLAS       MONTRÉAL   
4       Habitat for Humanity Manitoba (Winkler-Morden ...       Winnipeg   
...            

In [165]:
# Step 6 - Identify and remove any gifts already uploaded into GC
# Only check against reference gifts from the same year as the donees data

ref_bn = reference['BN'].tolist()

# ~ = is not in, i.e. this is keeping only BNs not in ref_bn
donees = step_five[~step_five['BN'].isin(ref_bn)]

print(donees)


                     BN          DoneeBN                        DoneeName  \
11      101694578RR0001  119147072RR0001    SHAAREI SHOMAYIM CONGREGATION   
12      101694578RR0001  842008278RR0001                        THE HOUSE   
13      101694578RR0001  129950291RR0001       NETIVOT HATORAH DAY SCHOOL   
14      101694578RR0001  119043891RR0001               MIZRACHI OF CANADA   
15      101694578RR0001  863109757RR0001                     JACS TORONTO   
...                 ...              ...                              ...   
162401  899977177RR0001  870077757RR0001                 MARGARET'S HOUSE   
162402  899977177RR0001  107461485RR0001                 OPEN DOOR CENTRE   
162403  899977177RR0001  119108421RR0001       OUR DAILY BREAD MINISTRIES   
162404  899977177RR0001        110245611             MISSION TO SEAFARERS   
162405  899977177RR0001  118801448RR0001  BEACON HOUSE INTERFAITH SOCIETY   

                 City ReportedAmt  Year Purpose  
11            TORONTO    

In [166]:
# Update the Master Reference Table - Gift Data sheet in the Gift Data Tracking workbook with new gifting funders this year
# Might want to create a pop-up notification here to remind user to update with this value

ref_update = pd.DataFrame(masterlist, columns= ['nid', 'uuid', 'FoundationTitle', 'BN'])
new_donees = pd.DataFrame(donees, columns= ['BN'])
ref_update = pd.merge(new_donees, ref_update, on ='BN', how ='inner')
ref_update = ref_update.drop_duplicates('BN')
print(ref_update)

# *UPDATE HERE* - Update path to your own local folder, and remember to use this CSV to update the Gift Data Tracking workbook
# *UPDATE GIFT TRACKING*
ref_update.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\step6_update_tracking.csv', 
                   encoding = 'ANSI', index = False, header=True)


                    BN    nid                                  uuid  \
0      101694578RR0001  79552  e751a107-04cc-48e4-84fb-0402ba6e869d   
12     101835163RR0001  85253  d67b3bf3-e8d5-4a62-8582-20fdc1461e95   
13     101835221RR0001  79281  bb01d2b3-ff65-4c87-aa90-83527d2f2ca3   
24     101835445RR0001  85292  de0bf8a4-fab2-496d-8dd6-547f62a1dfc5   
25     101835486RR0001  79548  e79b0c20-c98d-4a8c-88aa-83475cfd5ad6   
...                ...    ...                                   ...   
82968  899779441RR0001  86085  f0dbdfb7-3aa6-442d-926d-b7d1ebed0519   
82969  899791347RR0001  86509  78ed89d7-22c8-4c46-8e2b-3101da287326   
82970  899899660RR0001  81613  54595c69-5d08-471e-babb-a73d71aad508   
83010  899946115RR0001  80722  3f22d0f7-97e3-4bf7-a44d-d4e50f6569f0   
83039  899977177RR0001  87119  fa396755-6305-415a-9db4-d8053894c5a9   

                                       FoundationTitle  
0      Eugene and Lilly Sandorfy Charitable Foundation  
12       Fondation Pour La Santé 

In [190]:
# Step 7 - Foundation Activity
# Create a subset of only foundations to work with
activity = masterlist[masterlist['FoundationCategory'] == "Foundations"]
activity = pd.DataFrame(masterlist, columns= ['BN', 'FoundationActivity'])

# Count by foundation activity
step_seven = pd.merge(donees, activity, on ='BN', how ='left')

# *UPDATE GIFT TRACKING*
# Update the Master Reference Table - Gift Data sheet in the Gift Data Tracking workbook with number of grantmaking foundations
# Add up all the counts of grantmaking values
step_seven['FoundationActivity'].value_counts()

Grantmaking                                                                          36276
Administration of Donor Advised Funds                                                14659
No Activity, Fundraising Intermediary                                                11112
Administration of Donor Advised Funds, Grantmaking                                    5642
Fundraising Intermediary                                                              3747
Unclassified                                                                          2576
Administration of Donor Advised Funds, Grantmaking, Direct Charitable Programming     1952
Grantmaking, Direct Charitable Programming                                            1791
Direct Charitable Programming                                                         1142
Grantmaking, Administration of Donor Advised Funds                                     923
No Activity                                                                            867

In [168]:
# Step 8
charities.rename(columns = {"BN/Registration number:": "BN"}, inplace = True)

cra_bn = charities['BN'].tolist()

# Keep only BNs that are in the Charities - All dataset
donees = donees[donees['BN'].isin(cra_bn)]

# Reset index of rows
donees.reset_index(drop=True, inplace=True)
print(donees)

                    BN          DoneeBN                        DoneeName  \
0      101694578RR0001  119147072RR0001    SHAAREI SHOMAYIM CONGREGATION   
1      101694578RR0001  842008278RR0001                        THE HOUSE   
2      101694578RR0001  129950291RR0001       NETIVOT HATORAH DAY SCHOOL   
3      101694578RR0001  119043891RR0001               MIZRACHI OF CANADA   
4      101694578RR0001  863109757RR0001                     JACS TORONTO   
...                ...              ...                              ...   
83043  899977177RR0001  870077757RR0001                 MARGARET'S HOUSE   
83044  899977177RR0001  107461485RR0001                 OPEN DOOR CENTRE   
83045  899977177RR0001  119108421RR0001       OUR DAILY BREAD MINISTRIES   
83046  899977177RR0001        110245611             MISSION TO SEAFARERS   
83047  899977177RR0001  118801448RR0001  BEACON HOUSE INTERFAITH SOCIETY   

                City ReportedAmt  Year Purpose  
0            TORONTO     $500.00  2019

In [169]:
# Count by foundation activity again
grantmaking_foundations = pd.merge(donees, activity, on ='BN', how ='left')
print(grantmaking_foundations)

# *UPDATE GIFT TRACKING*
# Update the Master Reference Table - Gift Data sheet in the Gift Data Tracking workbook with number of grantmaking foundations
# Add up all the counts of grantmaking values
grantmaking_foundations['FoundationActivity'].value_counts()

                    BN          DoneeBN                        DoneeName  \
0      101694578RR0001  119147072RR0001    SHAAREI SHOMAYIM CONGREGATION   
1      101694578RR0001  842008278RR0001                        THE HOUSE   
2      101694578RR0001  129950291RR0001       NETIVOT HATORAH DAY SCHOOL   
3      101694578RR0001  119043891RR0001               MIZRACHI OF CANADA   
4      101694578RR0001  863109757RR0001                     JACS TORONTO   
...                ...              ...                              ...   
83045  899977177RR0001  870077757RR0001                 MARGARET'S HOUSE   
83046  899977177RR0001  107461485RR0001                 OPEN DOOR CENTRE   
83047  899977177RR0001  119108421RR0001       OUR DAILY BREAD MINISTRIES   
83048  899977177RR0001        110245611             MISSION TO SEAFARERS   
83049  899977177RR0001  118801448RR0001  BEACON HOUSE INTERFAITH SOCIETY   

                City ReportedAmt  Year Purpose             FoundationActivity  
0      

Grantmaking                                                                          36277
Administration of Donor Advised Funds                                                14661
No Activity, Fundraising Intermediary                                                11112
Administration of Donor Advised Funds, Grantmaking                                    5642
Fundraising Intermediary                                                              3747
Unclassified                                                                          2576
Administration of Donor Advised Funds, Grantmaking, Direct Charitable Programming     1952
Grantmaking, Direct Charitable Programming                                            1791
Direct Charitable Programming                                                         1142
Grantmaking, Administration of Donor Advised Funds                                     923
No Activity                                                                            867

In [170]:
# Step 9 - Check length of BNs, missing RR0001, etc.

donees['DoneeBN_len'] = donees['DoneeBN'].str.len()
print(donees)

# Just for QA'ing the code:
# bn_check = donees[donees['DoneeBN_len'] < 15]
# bn_check.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\bn_check.csv', 
#                   encoding = 'ANSI', index = False, header=True)

                    BN          DoneeBN                        DoneeName  \
0      101694578RR0001  119147072RR0001    SHAAREI SHOMAYIM CONGREGATION   
1      101694578RR0001  842008278RR0001                        THE HOUSE   
2      101694578RR0001  129950291RR0001       NETIVOT HATORAH DAY SCHOOL   
3      101694578RR0001  119043891RR0001               MIZRACHI OF CANADA   
4      101694578RR0001  863109757RR0001                     JACS TORONTO   
...                ...              ...                              ...   
83043  899977177RR0001  870077757RR0001                 MARGARET'S HOUSE   
83044  899977177RR0001  107461485RR0001                 OPEN DOOR CENTRE   
83045  899977177RR0001  119108421RR0001       OUR DAILY BREAD MINISTRIES   
83046  899977177RR0001        110245611             MISSION TO SEAFARERS   
83047  899977177RR0001  118801448RR0001  BEACON HOUSE INTERFAITH SOCIETY   

                City ReportedAmt  Year Purpose  DoneeBN_len  
0            TORONTO     

In [171]:
# Check if DoneeBNs have letters in them

#donees['contains_letters'] = donees['DoneeBN'].str.extract(pat ='([a-zA-Z])') - just for reference
donees['contains_letters'] = donees['DoneeBN'].str.findall("[a-zA-Z]")


In [172]:
# Deal with different types of BN problems
# Start with easy DoneeBNs first
# 1. 9 characters, all numeric digits -> Just add RR0001 to the end
# 2. 11 characters, ends in RR and contains_letters = ['R','R'] -> Add 0001 to the end

# Add a column that is True if the only letters in DoneeBN are RR, and False otherwise
donees['RR'] = donees['contains_letters'].apply(lambda x: x==['R', 'R'])
donees['rr'] = donees['contains_letters'].apply(lambda x: x==['r', 'r'])

# Convert all boolean to string so we can use it in our np.select below
mask = donees.applymap(type) != bool
d = {True: 'TRUE', False: 'FALSE'}
donees = donees.where(mask, donees.replace(d))

# Apply conditions using np.select
cond_12 = [(donees.DoneeBN_len == 9) & (donees.contains_letters.str.len() == 0), 
          (donees.DoneeBN_len == 11) & donees.DoneeBN.str.endswith("RR") & (donees.RR == "TRUE")]
choices_12 = [donees.DoneeBN + "RR0001", donees.DoneeBN + "0001"]

donees['DoneeBN'] = np.select(cond_12, choices_12, donees.DoneeBN)

print(donees[donees['DoneeBN_len'] == 9])

# Checkpoint - Check that RR0001 and 0001 were added correctly
# donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\9digits.csv', 
#                   encoding = 'ANSI', index = False, header=True)

                    BN          DoneeBN  \
137    104268586RR0001  118918804RR0001   
138    104268586RR0001  118808914RR0001   
141    104268586RR0001  119307577RR0001   
142    104268586RR0001  107690885RR0001   
1776   118839182RR0001  899728133RR0001   
...                ...              ...   
81874  896713500RR0001  132286808RR0001   
82741  898780721RR0001  136491875RR0001   
82875  898887005RR0001  119047959RR0001   
82913  898975958RR0001  827451998RR0001   
83046  899977177RR0001  110245611RR0001   

                                      DoneeName      City  ReportedAmt  Year  \
137                   FIRST PRESBYTERIAN CHURCH   PORTAGE    $3,000.00  2019   
138                     BIG BROTHER BIG SISTERS   PORTAGE    $3,000.00  2019   
141                                         YFC   PORTAGE    $3,000.00  2019   
142                                MCC MANITOBA  WINNIPEG  $308,000.00  2019   
1776                   CARMAN MEMORIAL HOSPITAL    CARMAN    $6,832.00  2019   
...

In [173]:
# 3. Replace rr with RR in DoneeBN where contains_letters = ['r', 'r'] in 15-character BNs

donees['DoneeBN'] = np.where((donees['rr'] == 'TRUE') & (donees['DoneeBN_len'] == 15),
                       donees['DoneeBN'].str.replace(r'\drr\d','RR', regex = True),
                       donees['DoneeBN'])

# 4. Delete: 15 characters where contains_letters != ['R', 'R'] or ['r', 'r']

cond_4 = [(donees.DoneeBN_len == 15) & (donees.RR == "FALSE") & (donees.rr == "FALSE")]
choices_4 = [""]
donees['DoneeBN'] = np.select(cond_4, choices_4, donees.DoneeBN)

print(donees[donees['DoneeBN_len'] == 15])

                    BN          DoneeBN                        DoneeName  \
0      101694578RR0001  119147072RR0001    SHAAREI SHOMAYIM CONGREGATION   
1      101694578RR0001  842008278RR0001                        THE HOUSE   
2      101694578RR0001  129950291RR0001       NETIVOT HATORAH DAY SCHOOL   
3      101694578RR0001  119043891RR0001               MIZRACHI OF CANADA   
4      101694578RR0001  863109757RR0001                     JACS TORONTO   
...                ...              ...                              ...   
83041  899977177RR0001  871610341RR0001                  CAMP EVANGELINE   
83043  899977177RR0001  870077757RR0001                 MARGARET'S HOUSE   
83044  899977177RR0001  107461485RR0001                 OPEN DOOR CENTRE   
83045  899977177RR0001  119108421RR0001       OUR DAILY BREAD MINISTRIES   
83047  899977177RR0001  118801448RR0001  BEACON HOUSE INTERFAITH SOCIETY   

                City ReportedAmt  Year Purpose  DoneeBN_len contains_letters  \
0      

In [174]:
# 5. 14 characters
# Find and replace RR001 with RR0001, RR000 with RR0001
# Find and replace 0R0001 with 0RR0001, 1R0001 with 1RR0001, etc.
cond_5 = [(donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("RR001")), 
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("RR000")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("0R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("1R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("2R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("3R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("4R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("5R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("6R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("7R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("8R0001")),
          (donees.DoneeBN_len == 14) & (donees.DoneeBN.str.endswith("9R0001"))]
choices_5 = [donees.DoneeBN.str.replace("RR001", "RR0001"), 
            donees.DoneeBN.str.replace("RR000", "RR0001"),
            donees.DoneeBN.str.replace("0R0001", "0RR0001"),
            donees.DoneeBN.str.replace("1R0001", "1RR0001"), 
            donees.DoneeBN.str.replace("2R0001", "2RR0001"),
            donees.DoneeBN.str.replace("3R0001", "3RR0001"),
            donees.DoneeBN.str.replace("4R0001", "4RR0001"),
            donees.DoneeBN.str.replace("5R0001", "5RR0001"), 
            donees.DoneeBN.str.replace("6R0001", "6RR0001"),
            donees.DoneeBN.str.replace("7R0001", "7RR0001"),
            donees.DoneeBN.str.replace("8R0001", "8RR0001"),
            donees.DoneeBN.str.replace("9R0001", "9RR0001")]

donees['DoneeBN'] = np.select(cond_5, choices_5, donees.DoneeBN)
print(donees[donees['DoneeBN_len'] == 14])

# Checkpoint
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\14digits.csv', 
#                   encoding = 'ANSI', index = False, header=True)

                    BN          DoneeBN  \
337    106885312RR0001   11914017RR0001   
421    106885312RR0001   10745382RR0001   
642    107641953RR0001   14035108RR0001   
708    107641953RR0001   11924065RR0001   
726    107641953RR0001   88909719RR0001   
...                ...              ...   
80273  896713500RR0001   108071671RR003   
81966  896713500RR0001  118831106RR0001   
82595  898081757RR0001   11882661RR0001   
83033  899946115RR0001  103115812RR0001   
83035  899946115RR0001  106845100RR0001   

                                       DoneeName           City ReportedAmt  \
337    LES AINÉS ET LES AINÉES DE JONQUIÈRE INC.      JONQUIÈRE     $197.00   
421                 GROUPE AIDE-ACTION ST-HONORÉ   SAINT-HONORÉ  $10,000.00   
642               TETRA SOCIETY OF NORTH AMERICA      VANCOUVER     $850.00   
708                  LONDON PUBLIC LIBRARY BOARD         LONDON   $8,175.00   
726                     L'ARCHE CANADA FOUNDTION  RICHMOND HILL     $250.00   
...      

In [175]:
# 6. Where contains_letters = ["R"] and DoneeBN_len = 14, replace R with RR

# Add a column that is True if the only letter in DoneeBN is one R, and False otherwise
donees['just_R'] = donees['contains_letters'].apply(lambda x: x==['R'])

# Convert boolean to string so we can use it in our np.select below
mask = donees.applymap(type) != bool
d = {True: 'TRUE', False: 'FALSE'}
donees = donees.where(mask, donees.replace(d))

# Replace the R with RR following Condition #6
donees['DoneeBN'] = np.where((donees['just_R'] == 'TRUE') & (donees['DoneeBN_len'] == 14),
                       donees['DoneeBN'].str.replace(r'\dR\d','RR', regex = True),
                       donees['DoneeBN'])

# Checkpoint - Check that 14-character DoneeBNs with R now have RR instead 
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\replaceR.csv', 
#                   encoding = 'ANSI', index = False, header=True)

In [176]:
# Calculate contains_letters and DoneeBN_len again, now that DoneeBN has been updated
donees['DoneeBN_len'] = donees['DoneeBN'].str.len()
donees['contains_letters'] = donees['DoneeBN'].str.findall("[a-zA-Z]")

# Checkpoint - check that the new columns have the right values
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\bn_check2.csv', 
#                   encoding = 'ANSI', index = False, header=True)

In [177]:
# Calculate contains_letters and DoneeBN_len again, now that DoneeBN has been updated
donees['DoneeBN_len'] = donees['DoneeBN'].str.len()
donees['contains_letters'] = donees['DoneeBN'].str.findall("[a-zA-Z]")

# Checkpoint - check that no 15-character DoneeBNs have any letters other than RR
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\condition7.csv', 
#                   encoding = 'ANSI', index = False, header=True)

In [178]:
# 8. Delete DoneeBN where length < 15
cond_8 = [donees.DoneeBN_len < 15]
choices_8 = [""]
donees['DoneeBN'] = np.select(cond_8, choices_8, donees.DoneeBN)

# Checkpoint - only 15-character DoneeBNs should remain
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\final_15digit.csv', 
#                   encoding = 'ANSI', index = False, header=True)
print(donees[donees['DoneeBN_len'] < 15])

                    BN DoneeBN                                  DoneeName  \
337    106885312RR0001          LES AINÉS ET LES AINÉES DE JONQUIÈRE INC.   
421    106885312RR0001                       GROUPE AIDE-ACTION ST-HONORÉ   
642    107641953RR0001                     TETRA SOCIETY OF NORTH AMERICA   
708    107641953RR0001                        LONDON PUBLIC LIBRARY BOARD   
726    107641953RR0001                           L'ARCHE CANADA FOUNDTION   
...                ...     ...                                        ...   
80241  896713500RR0001                      THE CONCORDIA FOUNDATION INC.   
80272  896713500RR0001                               THE ARTS ACCESS FUND   
80273  896713500RR0001                   THE ARTHRITIS SOCIETY - BC/YUKON   
80318  896713500RR0001                       STRONGEST FAMILIES INSTITUTE   
82595  898081757RR0001                          DOUPHIN FRIENDSHIP CENTER   

                  City ReportedAmt  Year Purpose  DoneeBN_len  \
337       

In [179]:
# Step 10 - Remove gift records missing both DoneeName and DoneeBN

donees.drop(donees[(donees['DoneeName'].isnull()) & (donees['DoneeBN'].isnull())].index, inplace = True)

# Check that the correct gift records were deleted
#donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\nonameBN.csv', 
#                   encoding = 'ANSI', index = False, header=True)

In [180]:
# Step 12 before 11 since part of it can be automated
# Just create a column that marks all rows with "see attached, qualified donee, voir liste, etc."
# Then export to csv and manually delete them based on judgement

# Add column that is TRUE if DoneeName contains "see attached, qualified donee, voir liste, etc."
# (?i) makes regex Case Insensitive
donees['bad_Name'] = donees['DoneeName'].str.contains("attached|qualified donee|voir la liste|voir liste(?i)")
# Add column that is TRUE if ReportedAmt contains any letters/words
donees['bad_Amt'] = donees['ReportedAmt'].str.contains("[a-zA-Z]")

# Convert boolean to string so we can print the dataframe here
mask = donees.applymap(type) != bool
d = {True: 'TRUE', False: 'FALSE'}
donees = donees.where(mask, donees.replace(d))

print(donees[donees['bad_Amt'] == "TRUE"])

# Delete unnecessary columns
donees.drop(['DoneeBN_len', 'contains_letters', 'RR', 'rr', 'just_R'], axis=1, inplace=True)

# Step 11 - Remove gift records with ReportedAmt < $0
# To be done manually since, in a few instances, a funder may report ALL of their gifts as a deductible/negative value.
# In that case, the gifts need to be converted to positive values.

# Upon export, remove gifts with ReportedAmt < $0, then remove gifts with bad_Name or bad_Amt = TRUE (if it makes sense to)
# *UPDATE HERE* - Update path to your own local folder
donees.to_csv (r'C:\Users\Catherine\Documents\Imagine Canada\Gift Import Cleaning\readyforStep11.csv', 
                   encoding = 'ANSI', index = False, header=True)

Empty DataFrame
Columns: [BN, DoneeBN, DoneeName, City, ReportedAmt, Year, Purpose, DoneeBN_len, contains_letters, RR, rr, just_R, bad_Name, bad_Amt]
Index: []
